# TP4-NMF-Bejaoui Ahmed et Mejri Aymen

## Les imports nécéssaires pour le tp

In [1]:
from __future__ import print_function
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, \
CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
import numpy as np

### Question 1 : 

In [2]:
def FtoV(_vectorizer=TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english'), _random_state=1):
    vectorizer = _vectorizer
    random_state = _random_state
    # Fetch data and vectorize
    print("Loading dataset...")
    dataset = fetch_20newsgroups(shuffle=True, random_state=random_state,
                                 remove=('headers', 'footers', 'quotes'))
    data_samples = dataset.data[:2000]        
    t0 = time()
    features = vectorizer.fit_transform(data_samples)
    feature_names = vectorizer.get_feature_names()
    print("done in %0.3fs." % (time() - t0))
    return features, feature_names

In [3]:

def  NMFModel (features, _vectorizerName='tf_idf', _random_state=1, 
             _beta_loss=None, _init='random', _W=None, _H=None, _K = None):
    
    n_samples = 2000
    n_features = 1000
    n_top_words = 20
    n_components = 10 if _K is None else _K
    vectorizerName = _vectorizerName
    random_state = _random_state
    solver = 'cd' if _beta_loss is None else 'mu'
    beta_loss = 'frobenius' if _beta_loss is None else _beta_loss
    init = _init
    print("Fitting the NMF model ("+beta_loss+" norm) with "+str(vectorizerName)+" features, "
       "n_samples=%d and n_features=%d..." % (n_samples, n_features))
    
    t0 = time()
#     if _init is None:
#         nmf = NMF(n_components=n_components, 
#                   random_state=_random_state,
#                   solver = solver,
#                   beta_loss = beta_loss,
#                   alpha=.1, l1_ratio=.5).fit(features)
#     else:
    nmf = NMF(n_components=n_components, 
              random_state=_random_state,
              solver = solver,
              beta_loss = beta_loss,
              alpha=.1, l1_ratio=.5)
    nmf.fit_transform(features, W=_W, H=_H)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model ("+beta_loss+" norm):")
    return nmf, n_top_words

In [4]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [5]:
def runs(_vectorizer=None, _vectorizerName=None, _random_state=None, _beta_loss=None, 
               _init='random', _W=None, _H=None, _K=None,dprint=True):
    features, feature_names = FtoV()
    nmf, n_top_words = NMFModel(features, _vectorizerName, _random_state, _beta_loss, _init, _W, _H, _K)
    print('nombre d\'itération pour la convergence : ',str(nmf.n_iter_))
    print("error : ",nmf.reconstruction_err_,' \n')
    if(dprint):
        print_top_words(nmf, feature_names, n_top_words)
    print('#########################################################################')


In [6]:
init=['nndsvda','nndsvdar','random']
k=20
for i in init:
    runs(_random_state=k,_init=i)
    k=k+20


Loading dataset...
done in 0.311s.
Fitting the NMF model (frobenius norm) with None features, n_samples=2000 and n_features=1000...
done in 0.230s.

Topics in NMF model (frobenius norm):
nombre d'itération pour la convergence :  134
error :  42.1386080031  

Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope

**interprétation :**
<br> nous avons essayé 3 configurations d'initialisation possibles pour le "init". la première c'est le nndsva qui permet de faire une double décomposition des valeurs singulières non-négatives sur les features tout en remplaçant les valeurs négatives en 0. Cette méthode converge avec le minimum d'iteration avec un taux d'erruer minimale. Les autres initialisation (nndsvar et random) convergent moin rapidement avec un pourcentage d'erreur qui est semblable au "nndsvda

### Question 2 :

In [7]:
beta=['frobenius','kullback-leibler']
k=20
for i in beta:
    runs(_random_state=k,_beta_loss=i)
    k=k+20

Loading dataset...
done in 0.371s.
Fitting the NMF model (frobenius norm) with None features, n_samples=2000 and n_features=1000...
done in 0.062s.

Topics in NMF model (frobenius norm):
nombre d'itération pour la convergence :  30
error :  42.1877454643  

Topic #0: just people don like think know good time make way really right say ve want did new ll government use
Topic #1: windows use dos using window help card program software pc drivers os video application looking running version files screen ftp
Topic #2: god jesus bible faith christian christ does christians heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know mail does advance interested info hi email anybody list like send new address post appreciated reply looking information
Topic #4: 00 sale 10 condition price card new offer car 250 asking 15 12 today 20 50 11 cd 25 30
Topic #5: edu soon com university internet send mit cc ftp article information hope need email pub home mail b

**Interprétation :** <br>
lorsque nous utilsons le "Kullback-Lieber cost", Le NMF devient moins performant qu'avec le " $ l_{2} $ cost ". En fait, il lui faut 3 fois plus de temps pour converger et il tend à faire plus d'erreur.

De plus, les topics trouvés avec les deux fonctions couts semblent être identiques(religion, programmation, jeux...). Par contre, on peut remarque que le "l2 costs" produit des résultats plus précis puisque tous les mots qui sont regroupés dans le même topic sont plus représentatives et ont plus de sens par rapport à celle trouvé par le kullback-leibler cost

### Question 3 :

In [8]:
vectname=['TermFrequencyVectorizer','CountVectorizer']
for i in vectname:
    runs(_random_state=k,_vectorizerName=i)
    k=k+20

Loading dataset...
done in 0.310s.
Fitting the NMF model (frobenius norm) with TermFrequencyVectorizer features, n_samples=2000 and n_features=1000...
done in 0.217s.

Topics in NMF model (frobenius norm):
nombre d'itération pour la convergence :  128
error :  42.1386080329  

Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub artic

** Interprétation : ** 

Bien que le modèle utilisant la représentation term-frequency soit plus rapide avec un nombre d'itérations plus faible que le modèle utilisant le TF-IDF, il a une erreur plus importante.
    
Ceci pourrait etre dû au fait que les méthodes TF-IDF ont de nombreux avantages par rapport à la représentation TF:

La fréquence-terme (TF) n'est qu'une partie de l'approche TF-IDF pour la recherche d'information. L'autre partie est la fréquence de document inverse (IDF) qui est une mesure de la rareté d'un terme.

Il fournit une métrique de base pour extraire les termes les plus descriptifs d'un document et nous pouvons donc facilement calculer la similarité entre deux documents qui l'utilisent.

Il est facile de remarquer que les résultats avec TF-IDF sont plus appropriés.

## Partie 2 :

In [11]:
def puissance(w,h,k):
    """
    input:
    w: matrice
    h: matrice
    k: ordre de la puissance
    
    
    
    output:
    x: (w*h)**k
    
    
    """
    x=np.dot(w,h)
    if((k)==-1):
        x=np.divide(np.ones(x.shape),x)
    else:
        x=np.power(x,k)
    return x

def beta_divergence(V,W,H,beta,F,N,K):

    div = 0
    WH = np.dot(W, H)
    for i in range(F):
        for j in range(N):
                x = V[i][j] if V[i][j] != 0 else np.finfo(np.double).tiny
                y = WH[i][j]
                if beta == 1: 
                    div += x*np.log(x/y) - x + y
                elif beta == 0: 
                    div += (x/y) * np.log(x/y) - 1
                else: 
                    div += 1/(beta*(beta-1))*(pow(x,beta) + (beta-1)*pow(y,beta) - beta*x*pow(y,beta-1))
    return div
    
def NMF_function(V,W=None,H=None,beta=1,nbr_iter=20):
    """
    input:
    V:
    W: matrice W initiale(peut être non définie)
    H: matrice W initiale(peut être non définie)
    beta: coefficient de la beta divergence
    nbr_iter: nombre d'iteration
    
    
    
    output:
    W: matrice W predite
    H: matrice H predite
    
    
    """
    
    i=1
    eps=1
    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = np.random.rand(F,10)
        
    if H is None:
        H = np.random.rand(10,N)
    diff=1
    
    while(i<nbr_iter):
        A = np.dot(W.T, np.multiply(puissance(W,H,beta-2), V))
        B = np.dot(W.T, puissance(W,H,beta-1))
        Matrix_div = A / B
        H = np.multiply(H, Matrix_div)
        
        A = np.dot(np.multiply(puissance(W,H,beta-2), V),H.T)
        B = np.dot(puissance(W,H,beta-1), H.T)
        Matrix_div = A / B
        W = np.multiply(W, Matrix_div)
        
        matrix=np.dot(W,H)
        diff=np.linalg.norm(V-matrix)
        init_error = beta_divergence(V,W,H,beta,F,N,10)
        print('l\'erreur à l\'intération ',i,'est :',init_error)
        i+=1
    
    return W,H

In [12]:
features, feature_names = FtoV()

V1 = np.random.rand(features.shape[0], features.shape[1])
V1 = np.array(V1) # Data matrix F x N 
F = len(V1) #Number of V rows
N = len(V1[0])
W = np.random.rand(F,10)
H = np.random.rand(10,N)
W,H=NMF_function(V1,W,H)
print('la matrice W est : ',W)
print('la matrice H est : ',H)

Loading dataset...
done in 0.387s.
l'erreur à l'intération  1 est : 198383.022059
l'erreur à l'intération  2 est : 197721.250885
l'erreur à l'intération  3 est : 197162.163781
l'erreur à l'intération  4 est : 196683.606322
l'erreur à l'intération  5 est : 196269.382964
l'erreur à l'intération  6 est : 195907.339275
l'erreur à l'intération  7 est : 195588.179304
l'erreur à l'intération  8 est : 195304.674164
l'erreur à l'intération  9 est : 195051.11917
l'erreur à l'intération  10 est : 194822.9532
l'erreur à l'intération  11 est : 194616.486524
l'erreur à l'intération  12 est : 194428.70274
l'erreur à l'intération  13 est : 194257.112318
l'erreur à l'intération  14 est : 194099.642751
l'erreur à l'intération  15 est : 193954.555067
l'erreur à l'intération  16 est : 193820.379611
l'erreur à l'intération  17 est : 193695.866106
l'erreur à l'intération  18 est : 193579.944429
l'erreur à l'intération  19 est : 193471.693499
la matrice W est :  [[ 0.64331309  0.84075369  0.31521759 ...,  0.